# The functionality of ORC is presented in this notebook

First the wind turbine and farm details have to be initialized, here the example reference wind farm is used

But any other values for the wind turbine can be used here. Rated power has to be input in kilowatt (*kW*), rotor diameter and hub height in meters (*m*) !!

In [1]:
# The used wind turbine and farm is imported
from py_wake.examples.data.iea37._iea37 import IEA37_WindTurbines, IEA37Site

ws = IEA37Site().default_ws
n_t = 16
power_rated = int(IEA37_WindTurbines().power(ws)) / 1000
rotor_diameter = int(IEA37_WindTurbines().diameter())
hub_height = int(IEA37_WindTurbines().hub_height())


The country in which the wind turbine is to be built needs to be specified, here the example country is France

In [2]:
country = 'FRA'

# Cost component estimation

CAPEX estimation using the reference wind farm

In [6]:
# Import the Python class for CAPEX estimation
from CapEx import CAPEX

# Initialize the turbine
TurbineCAPEX = CAPEX(rotor_diameter, power_rated, hub_height, country)

# Based on the choice of the user either the original or the LCOE-adjusted cost model is used
version = 'original'

if version == 'original':
    CAPEXcost = CAPEX.getCAPEX(TurbineCAPEX)
else:
    CAPEXcost = CAPEX.getCAPEXadjusted(TurbineCAPEX)

# The CAPEX 
print(CAPEXcost)

c:\Users\maxik\Desktop\ETH\Bachelorarbeit\Code\LCOE_model\CapEx.py:87: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv("databases\oww3.csv")


5963850.7115573995


OPEX estimation using the reference wind turbine

In [3]:
# Import the Python class for OPEX estimation
# Also import the Python classes that estimate the discount rate and AEP
from OpEx import OPEX
from DiscountRate import discountRate
from AEP_estimation import AEP

# Estimate the discount rate
d = discountRate.InfAdjRate(country)

# Estimate the AEP for the reference wind turbine
# (CHANGE ACCORDING TO THE PROJECT SPECIFIC-VALUE)
AEPvalue = AEP.AEP_sim()

# Initialize the turbine
TurbineOPEX = OPEX(rotor_diameter, power_rated, hub_height, d, AEPvalue, country)

# Based on the choice of the user either the original or the LCOE-adjusted cost model is used
version = 'original'

if version == 'original':
    OPEXcost = OPEX.getOPEX(TurbineOPEX)
else:
    OPEXcost = OPEX.getOPEXadjusted(TurbineOPEX)

# The annual OPEX
print(OPEXcost)


288764


Decommissioning cost estimation using the reference wind turbine

In [4]:
# Import the Python class for decommissioning cost estimation
from Decommissioning import EOL

# Initialize the turbine
TurbineDEC = EOL(rotor_diameter, power_rated, hub_height, country, n_t)

# The decommissioning cost estimation is the same for both model versions
DECcost = EOL.decommissioningCost(TurbineDEC)

# The decommissioning cost
print(DECcost)

-183488.0614


The LCOE gets calculated here using the above described cost, but there is also an overall LCOE estimation method without these steps, as this is supposed to show how it is computed and each cost component.

In [8]:
# Math module gets imported
import math

# Here the final LCOE calculation gets initialized
lifetime = 20
discountedEnergy = 0
discountedCost = 0
x = 0
# The discountedCost and discountedEnergy get calculated
while x <= lifetime:
    if x == 0:
        # CAPEXcost is added to LCOE equation before the start of operation 
        discountedCost += CAPEXcost
        x = x + 1
    elif x == 20:
        # OPEX costs get discounted and added to the LCOE equation
        discountedCost += (OPEXcost + DECcost) * 1 / (math.pow((1+d) , x))
        # Same happens to the energy produced annually
        discountedEnergy += (AEPvalue) * (1 / (math.pow((1+d) , x)))
        x = x + 1
    else:
        # OPEX costs get discounted and added to the LCOE equation
        discountedCost += OPEXcost * 1 / (math.pow((1+d) , x)) 
        # Same happens to the energy produced annually
        discountedEnergy += (AEPvalue) * (1 / (math.pow((1+d) , x)))
        x = x + 1

LCOEvalue = (discountedCost/discountedEnergy) * 1000
print(str(LCOEvalue) + " $/MWh")

32.443403805689854 $/MWh


# LCOE functionality

In this section the LCOE functionality is shown in one step without the previously described cost stages

In [1]:
# Again the reference wind turbine is initialized
# The used wind turbine and farm is imported
from py_wake.examples.data.iea37._iea37 import IEA37_WindTurbines, IEA37Site
from LCOE_final import LCOE

ws = IEA37Site().default_ws
n_t = 16
power_rated = int(IEA37_WindTurbines().power(ws)) / 1000
rotor_diameter = int(IEA37_WindTurbines().diameter())
hub_height = int(IEA37_WindTurbines().hub_height())
country = 'CZE'
version = 'original'

Turbine = LCOE(rotor_diameter, power_rated, hub_height, country, n_t, version)

LCOEvalue = LCOE.getLCOE(Turbine)

# LCOE value
print(LCOEvalue)

c:\Users\maxik\Desktop\ETH\Bachelorarbeit\Code\LCOE_model\CapEx.py:87: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv("databases\oww3.csv")


34.054134804249905
